In [1]:
import os, sys, json
sys.path.append('..')
import requests
from glob import glob
from pprint import pprint

from utils.file_process import read_base64
from utils.parse import parse_json

# FastAPI server URL
base_url = "http://127.0.0.1:3535/"

In [2]:
# OCR测试
ocr_url = os.path.join(base_url, 'ocr-info')

files_list = sorted( glob('../files/examples/ocr/*'))
files_payload = []

# 打开所有文件并添加到 payload 中
# for file_path in files_list:
    # file_obj = open(file_path, 'rb')
    # files_payload.append(('files', (os.path.basename(file_path), file_obj, 'application/octet-stream')))
    
# 发送 POST 请求
# JSON 数据
json_data = {
    "paths": json.dumps([tmp.replace('../','./') for tmp in files_list])
}
response = requests.post(ocr_url, data=json_data )   #files=files_payload)

ocr_dict_res = response.json()['ticket_data']
ocr_res = response.json()['raw_ocr_result']

In [3]:
response.json()


{'status': 20,
 'message': '[ocr-info] 完成',
 'ticket_data': {'工作负责人': '赵×',
  '工作班成员(不包括工作负责人)': ['王××', '赵x×', '田××', '李x×', '张×X'],
  '工作的线路名称或设备双重名称(多回路应注明双重称号)': '110kV桃源I1133线（右线，深蓝色）',
  '工作任务': '更换相悬垂线夹；更换三相绝缘子',
  '工作许可时间': '2023-10-10T08:45:00',
  '计划工作开始时间': '2023-10-10T08:30:00',
  '工作终结时间': '2023-10-10T15:45:00',
  '工作签发时间': '2023-10-09T08:20:00',
  '工作负责人人员签名': '赵X',
  '工作票延期': '无',
  '现场所挂的接地线编号': '01号、02号',
  '确认工作负责人布置的工作任务和安全措施工作班组人员签名': ['王××', '赵x×', '田××', '张x×'],
  '工作终结报告': '电话报告',
  '备注': '(1)指定专责监护人 李XX 负责监护王XX在7号塔更换A相悬垂线夹和9号更换三相绝缘子。负责监护赵XX在8号更换三相绝缘子。(2)其他事项',
  '工作票延期时间': '无',
  '工作负责人变动情况': '无'},
 'raw_ocr_result': '电 力 线 路 第 一 种 工 作 票   已 执 行   单 位 ： （ 松 ） 输 电 运 检 中 心   编 号 ： · 0 5 0 2 0 0 3 ·   1 . 工 作 负 责 人 （ 监 护 人 ） ： · · 赵 × . .   班 组 . 检 修 二 班 .   2 . 工 作 班 人 员 （ 不 包 括 工 作 负 责 人 ） ：   王 × × 、 赵 x × 、 田 × × 、 李 x × 、 张 × X   共 . . . 5 . . 人   3 . 工 作 的 线 路 或 设 备 双 重 名 称 （ 多 回 路 应 注 明 双 重 称 号 ）   1 1 0 k v 桃 源 I 1 1 3 3 线 （ 右 线 ， 深 蓝 色 ） ·   4 . 工 作 任 务   

In [4]:
# 逻辑测试
logic_url = os.path.join(base_url, 'logic-check')
payload = {'ticket_data':ocr_dict_res}

# 发送 POST 请求（也可以用 GET 依据实际需要）
response = requests.post(logic_url, json=payload)

flag, logic_json = parse_json( response.json()['logic_errors'])
pprint(logic_json)

{'不符合要求的内容': ['工作任务栏中缺少具体且明确的每段或每基杆塔的工作内容。',
              "工作班组人员签名中缺少'李x×'的签名，且多了一个'张x×'的签名。",
              '工作许可时间2023-10-10T08:45:00早于计划工作开始时间2023-10-10T08:30:00，未在备注中说明原因。']}


In [5]:
# 工人判定
worker_url = os.path.join(base_url, 'worker-check')
payload = {'ticket_data':ocr_dict_res}

# 发送 POST 请求（也可以用 GET 依据实际需要）
response = requests.post(worker_url, json=payload)
flag, worker_json = parse_json(response.json()['worker_errors'])
pprint(worker_json)

{'人数': '工作班成员不包含工作负责人个数为5，请检查人数是否一致',
 '工作班成员签名': '工作班人员签名与工作班人数不一致，但也有可能由于字迹潦草导致无法识别导致，请进行检查',
 '工作负责人签名': '工作负责人人员签名与工作班负责人不一致，但也有可能由于字迹潦草导致无法识别导致，请进行检查'}


In [6]:
response.json()


{'status': 20,
 'message': '完成',
 'worker_errors': {'人数': '工作班成员不包含工作负责人个数为5，请检查人数是否一致',
  '工作负责人签名': '工作负责人人员签名与工作班负责人不一致，但也有可能由于字迹潦草导致无法识别导致，请进行检查',
  '工作班成员签名': '工作班人员签名与工作班人数不一致，但也有可能由于字迹潦草导致无法识别导致，请进行检查'}}